In [ ]:
print("Hello World!!!")
!python --version

In [ ]:
klasses = ["toy_poodle", "border_collie", "pomeranian"]
num_klasses = len(klasses)
image_size = 50
num_testdata = 100

In [ ]:
import os, glob

from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np

X = []
Y = []

for index, klass in enumerate(klasses):
    files = glob.glob(f"./{klass}/*.jpg")
    for i, file in enumerate(files):
        if i >= 200:
            break
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)

        for angle in range(-20, 20, 5):
            img_r = image.rotate(angle)
            data = np.asarray(img_r)
            X.append(data)
            Y.append(index)

            img_trans = img_r.transpose(Image.FLIP_LEFT_RIGHT)
            data = np.asarray(img_trans)
            X.append(data)
            Y.append(index)

X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

np.savez("./dogs.npz", X_train=X_train, X_test=X_test, Y_train=Y_train, Y_test=Y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn import model_selection
from tensorflow.keras.optimizers import RMSprop


def main():
    data = np.load("./dogs.npz")
    X_train = data["X_train"]
    X_test = data["X_test"]
    Y_train = data["Y_train"]
    Y_test = data["Y_test"]

    X_train = X_train.astype("float") / 256
    X_test = X_test.astype("float") / 256
    Y_train = to_categorical(Y_train, num_klasses)
    Y_test = to_categorical(Y_test, num_klasses)

    model = model_train(X_train, Y_train)
    model_eval(model, X_test, Y_test)


def model_train(X_train, Y_train):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=X_train.shape[1:]))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation("softmax"))

    opt = RMSprop(learning_rate=0.0001, decay=1e-6)

    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.fit(X_train, Y_train, batch_size=32, epochs=100)

    model.save("./dogs.h5")

    return model


def model_eval(model, X_test, Y_test):
    scores = model.evaluate(X_test, Y_test, verbose=1)
    print("Test Loss: ", scores[0])
    print("Test Accuracy: ", scores[1])


if __name__ == "__main__":
    main()

In [ ]:
from keras.models import load_model
import sys
import numpy as np
from PIL import Image

def main():
    image = Image.open('image.jpeg')
    image = image.convert('RGB')
    image = image.resize((image_size, image_size))
    data = np.asarray(image) / 255
    X = []
    X.append(data)
    X = np.array(X)
    model = load_model('./dogs.h5')

    result = model.predict([X])[0]
    predicted = result.argmax()
    print({ "class": klasses[predicted], "probability": result[predicted] })

if __name__ == "__main__":
    main()